## Coursera Capstone Project Report
### The Battle of the Neighborhoods

In [10]:
import numpy as np # library to handle data in a vectorized manner
import time
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe


!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
import folium # map rendering library
from folium import plugins

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

import seaborn as sns

# import k-means from clustering stage
from sklearn.cluster import KMeans


print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.0

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.0

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Libraries imported.


In [11]:

address = 'Brooklyn, Newyork'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Brooklyn are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of Brooklyn are 40.6740635, -73.9862307.


In [12]:
neighborhood_latitude=40.6740635
neighborhood_longitude=-73.9862307

In [17]:
CLIENT_ID = 'M12NTNG0XL---------QZCTNJ0NG5RXKIFKMNLJ2W' # your Foursquare ID
CLIENT_SECRET = 'TSLNQ3---------W4WALGGWGZ5TVCD2WTDQYNF01T21FQCRQ' # your Foursquare Secret
VERSION = '20---04'
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

In [18]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [19]:
url= 'https://api.foursquare.com/v2/venues/explore?&client_id=M12NTNG0XL3O05IYMKRNIWFULVQZCTNJ0NG5RXKIFKMNLJ2W&client_secret=TSLNQ3OEHLKAWTHW4WALGGWGZ5TVCD2WTDQYNF01T21FQCRQ&v=20180604&ll=40.6740635,-73.9862307&radius=500&limit=100'

In [20]:
results = requests.get(url).json()

In [21]:
venues = results['response']['groups'][0]['items']
    
Bnearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
Bnearby_venues =Bnearby_venues.loc[:, filtered_columns]

# filter the category for each row
Bnearby_venues['venue.categories'] = Bnearby_venues.apply(get_category_type, axis=1)

# clean columns
Bnearby_venues.columns = [col.split(".")[-1] for col in Bnearby_venues.columns]

Bnearby_venues.head(10)

,name,categories,lat,lng
0,Artist & Craftsman Supply,Arts & Crafts Store,40.674341,-73.984297
1,J.J. Byrne Playground,Park,40.672939,-73.984085
2,Whole Foods Market,Grocery Store,40.675262,-73.988863
3,Washington Park,Park,40.672582,-73.986570
4,Dirty Precious,Cocktail Bar,40.676317,-73.986753
5,Russo's Mozzarella & Pasta,Deli / Bodega,40.673132,-73.983030
6,L'Albero dei gelati,Ice Cream Shop,40.671977,-73.983767
7,Culture: An American Yogurt Company,Frozen Yogurt Shop,40.672434,-73.983626
8,Baba's Pierogies,Eastern European Restaurant,40.676985,-73.986435
9,Stone Park Café,American Restaurant,40.672920,-73.983358


In [23]:
# create map of Brooklyn using latitude and longitude values
map_B = folium.Map(location=[neighborhood_latitude, neighborhood_longitude], zoom_start=20)

# add markers to map
for lat, lng, label in zip(Bnearby_venues['lat'], Bnearby_venues['lng'], Bnearby_venues['name']):
    label = folium.Popup(label, parse_html=True)
    folium.RegularPolygonMarker(
        [lat, lng],
        number_of_sides=4,
        radius=10,
        popup=label,
        color='blue',
        fill_color='#0f0f0f',
        fill_opacity=0.7,
    ).add_to(map_B)  
    
map_B

In [27]:
venue_id = '4fa862b3e4b0ebff2f749f06' # ID of Harry's Italian Pizza Bar
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/4fa862b3e4b0ebff2f749f06?client_id=M12NTNG0XL3O05IYMKRNIWFULVQZCTNJ0NG5RXKIFKMNLJ2W&client_secret=TSLNQ3OEHLKAWTHW4WALGGWGZ5TVCD2WTDQYNF01T21FQCRQ&v=20180604'

In [28]:
result = requests.get(url).json()
print(result['response']['venue'].keys())

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'price', 'hasMenu', 'likes', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'delivery', 'menu', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'hours', 'popular', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


In [29]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

6.8


In [30]:
venue_id = '4a08e2fff964a52022741fe3' # ID of Fiore Restaurant
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

7.9


In [31]:
venue_id = '4aa85f31f964a520075120e3' # ID of Vinny's of Carroll Gardens
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

8.0


##### Since this restaurant has a slightly better rating, let's explore it further.

In [32]:
result['response']['venue']['tips']['count']

39

In [33]:
## Vinny's of Carroll Gardens
limit = 15 # set limit to be greater than or equal to the total number of tips
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5df9fad954289056aa7d5f4e'},
 'response': {'tips': {'count': 39,
   'items': [{'id': '53095fbe11d2a9634edf062d',
     'createdAt': 1393123262,
     'text': 'Vinny is super friendly, been in the business 40 years! Food is great. No hipsters here, old school and neighborhoody.',
     'type': 'user',
     'canonicalUrl': 'https://foursquare.com/item/53095fbe11d2a9634edf062d',
     'lang': 'en',
     'likes': {'count': 1,
      'groups': [{'type': 'others',
        'count': 1,
        'items': [{'id': '10764674',
          'firstName': 'agnes',
          'lastName': 'chan',
          'gender': 'female',
          'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
           'suffix': '/MBGEXS0VI2BSRMH1.jpg'}}]}],
      'summary': '1 like'},
     'logView': True,
     'agreeCount': 3,
     'disagreeCount': 0,
     'todo': {'count': 0},
     'user': {'id': '9828894',
      'firstName': 'Krystal',
      'lastName': 'Solcich',
      'gender': 'female',

In [34]:
# Get tips and list of associated features
tips = results['response']['tips']['items']

tip = results['response']['tips']['items'][0]
tip.keys()

dict_keys(['id', 'createdAt', 'text', 'type', 'canonicalUrl', 'lang', 'likes', 'logView', 'agreeCount', 'disagreeCount', 'todo', 'user'])

In [35]:
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips) # json normalize tips

# columns to keep
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.gender', 'user.id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display tips
tips_filtered

,text,agreeCount,disagreeCount,id,user.firstName,user.lastName,user.gender,user.id
0,"Vinny is super friendly, been in the business 40 years! Food is great. No hipsters here, old school and neighborhoody.",3,0,53095fbe11d2a9634edf062d,Krystal,Solcich,female,9828894


In [36]:
latitude = 40.715337
longitude = -74.008848

In [37]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=M12NTNG0XL3O05IYMKRNIWFULVQZCTNJ0NG5RXKIFKMNLJ2W&client_secret=TSLNQ3OEHLKAWTHW4WALGGWGZ5TVCD2WTDQYNF01T21FQCRQ&ll=40.715337,-74.008848&v=20180604&radius=500&limit=100'

In [39]:
results = requests.get(url).json()
'There are {} around Vinnys of Carroll Gardens restaurant.'.format(len(results['response']['groups'][0]['items']))

'There are 100 around Vinnys of Carroll Gardens restaurant.'

In [40]:
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4af5d65ff964a52091fd21e3',
  'name': 'Korin',
  'location': {'address': '57 Warren St',
   'crossStreet': 'Church St',
   'lat': 40.71482437714839,
   'lng': -74.00940425461492,
   'labeledLatLngs': [{'label': 'display',
     'lat': 40.71482437714839,
     'lng': -74.00940425461492}],
   'distance': 73,
   'postalCode': '10007',
   'cc': 'US',
   'neighborhood': 'Tribeca',
   'city': 'New York',
   'state': 'NY',
   'country': 'United States',
   'formattedAddress': ['57 Warren St (Church St)',
    'New York, NY 10007',
    'United States']},
  'categories': [{'id': '4bf58dd8d48988d1f8941735',
    'name': 'Furniture / Home Store',
    'pluralName': 'Furniture / Home Stores',
    'shortName': 'Furniture / Home',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/furniture_',
     'suffix': '.png'},
    'primar

In [41]:
dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(10)

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,neighborhood,city,state,country,formattedAddress,id
0,Korin,Furniture / Home Store,57 Warren St,Church St,40.714824,-74.009404,"[{'label': 'display', 'lat': 40.71482437714839, 'lng': -74.00940425461492}]",73,10007,US,Tribeca,New York,NY,United States,"[57 Warren St (Church St), New York, NY 10007, United States]",4af5d65ff964a52091fd21e3
1,Chambers Street Wines,Wine Shop,148 Chambers St,btwn West Broadway & Hudson St,40.715773,-74.009718,"[{'label': 'display', 'lat': 40.715773063928374, 'lng': -74.00971823312332}]",88,10007,US,NaN,New York,NY,United States,"[148 Chambers St (btwn West Broadway & Hudson St), New York, NY 10007, United States]",4adcf23cf964a520cc6221e3
2,Juice Press,Vegetarian / Vegan Restaurant,83 Murray St,btwn Greenwich St & W Broadway,40.714788,-74.011132,"[{'label': 'display', 'lat': 40.71478769908051, 'lng': -74.0111317502157}]",202,10007,US,NaN,New York,NY,United States,"[83 Murray St (btwn Greenwich St & W Broadway), New York, NY 10007, United States]",54148bc6498ea7bb8c05b70a
3,Takahachi Bakery,Bakery,25 Murray St,at Church St,40.713653,-74.008804,"[{'label': 'display', 'lat': 40.713652845301894, 'lng': -74.0088038953017}]",187,10007,US,NaN,New York,NY,United States,"[25 Murray St (at Church St), New York, NY 10007, United States]",4c154c9a77cea593c401d260
4,Takahachi,Sushi Restaurant,145 Duane St,btwn W Broadway & Church St,40.716526,-74.008101,"[{'label': 'display', 'lat': 40.71652647412374, 'lng': -74.00810108466207}]",146,10013,US,NaN,New York,NY,United States,"[145 Duane St (btwn W Broadway & Church St), New York, NY 10013, United States]",4a8f2f39f964a520471420e3
5,Los Tacos No. 1,Taco Place,136 Church St,NaN,40.714267,-74.008756,"[{'label': 'display', 'lat': 40.714267, 'lng': -74.008756}]",119,10007,US,NaN,New York,NY,United States,"[136 Church St, New York, NY 10007, United States]",5d5f24ec09484500079aee00
6,Heyday,Spa,92 Reade St,NaN,40.715726,-74.007767,"[{'label': 'display', 'lat': 40.715726, 'lng': -74.007767}]",100,10013,US,NaN,New York,NY,United States,"[92 Reade St, New York, NY 10013, United States]",57ad129c498e05b086594d72
7,Philip Williams Posters,Antique Shop,122 Chambers St,NaN,40.715284,-74.008781,"[{'label': 'display', 'lat': 40.71528423132827, 'lng': -74.00878093952018}]",8,10007,US,NaN,New York,NY,United States,"[122 Chambers St, New York, NY 10007, United States]",4b747291f964a52042dd2de3
8,Whole Foods Market,Grocery Store,270 Greenwich Street,at Warren St,40.715579,-74.011368,"[{'label': 'display', 'lat': 40.715579155420606, 'lng': -74.01136823958119}]",214,10007,US,Tribeca,New York,NY,United States,"[270 Greenwich Street (at Warren St), New York, NY 10007, United States]",49bc3b0af964a52020541fe3
9,Equinox Tribeca,Gym,54 Murray St,at W Broadway,40.714099,-74.009686,"[{'label': 'display', 'lat': 40.71409860726041, 'lng': -74.0096857179283}]",154,10007,US,NaN,New York,NY,United States,"[54 Murray St (at W Broadway), New York, NY 10007, United States]",4a6e331af964a52031d41fe3


In [32]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


# add Ecco as a red circle mark
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Ecco',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map

### Explore Trending Venues

In [44]:
# define URL
url = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION)

# send GET request and get trending venues
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5df9fe60618f43001bd06020'},
 'response': {'venues': []}}

In [45]:
if len(results['response']['venues']) == 0:
    trending_venues_df = 'No trending venues are available at the moment!'
    
else:
    trending_venues = results['response']['venues']
    trending_venues_df = json_normalize(trending_venues)

    # filter columns
    columns_filtered = ['name', 'categories'] + ['location.distance', 'location.city', 'location.postalCode', 'location.state', 'location.country', 'location.lat', 'location.lng']
    trending_venues_df = trending_venues_df.loc[:, columns_filtered]

    # filter the category for each row
    trending_venues_df['categories'] = trending_venues_df.apply(get_category_type, axis=1)

In [46]:
# display trending venues
trending_venues_df

'No trending venues are available at the moment!'